# Дз 3. Тестирование статистических гипотез

<div class="alert alert-info" role="alert">

Важно!

* Плагиат карается обнулением работы
* При использовании чужого кода из открытых источников необходимо указать ссылку на источник
* В каждой задаче необходимо обосновывать выбор статистического критерия, формулировать нулевую и альтернативную гипотезу и делать интерпретацию получившихся результатов, а также содержательный вывод.
* Весь текст оформляется строго в ячейках типа markdown

## Задача 1. Осьминог Пауль (2 балла)


В период с 2008 по 2010 год в Германии жил Осьминог Пауль, который был известен тем, что мог предсказывать результаты матчей с участием сборной Германии.     
   
В таблице ниже вы можете видеть результаты его предсказаний: 


<a href="https://ibb.co/VWnc5yG"><img src="https://i.ibb.co/QFS3tBV/image.jpg" alt="image" border="0"></a>

Если считать, что случайным образом мы угадывали ли бы 50% матчей, то можно ли утверждать, что осьминог Пауль был талантливым экстрасенсом со способностями к предсказаниям?

In [5]:
import numpy as np
import pandas as pd
from scipy import stats
import scipy

Итак, чтобы проверить способности Пауля, предположим, что он угадывает исход случайным образом, то есть с вероятностью 0.5, тогда:

H0: Пауль угадывает случайным образом, p=1/2

H1: У Пауля есть способности к предсказанию, p>1/2

Для проверки нашей гипотезы используем биномиальный тест для оценки равенства вероятности заданной с числовыми данными


In [7]:
### ╰( ͡° ͜ʖ ͡° )つ──☆ сотворите статистическую магию
#  источник: семинар 6
result=stats.binom_test(11, 13, 0.5, alternative = 'greater')
print(f'p-value:{result}')


if result > 0.05:
    print('Пауль угадывает случайным образом, p=1/2')
else:
    print('У Пауля есть способности к предсказанию, p>1/2') 

p-value:0.01123046875
У Пауля есть способности к предсказанию, p>1/2


Мы получили значение p value = 0.01, из чего можно заключить, что на разумных уровнях значимости(5 и 10%) мы отвергаем нулевую гипотезу в пользу альтернативной. 
То есть Пауль на самом деле выдающийся предсказатель со способностями

## Задача 2.  Корм и котики (2 балла)

    
Новый корм для котиков рекламируется с помощью рекламного баннера. Отдел маркетинга разработал для рекламы новый баннер. Они считают, что он более красивый. Перед фирмой возникла необходимость проверить какой из двух баннеров лучше.

Для этого была создана специальная веб-форма. В этой веб-форме людям по очереди демонстрировали оба баннера и просили поставить лайк, если баннер нравится человеку. Если человеку нравятся оба баннера, он может поставить два лайка. Если ему не нравится ни один баннер, он может не ставить лайки вовсе. Собранные в конечном итоге данные находятся в табличке.

Можно ли утверждать, что оба баннера одинаково хороши? Сформулируйте статистическую гипотезу для проверки того, что оба баннера выбирают одинаково часто. Проверьте ее и сделайте выводы.


In [11]:
columns=['banner1', 'banner2']
df = pd.read_csv('banner_click_stat.csv',names=columns, sep='\t')
df.head()

,banner1,banner2
0,0,0
1,1,1
2,0,0
3,0,0
4,0,0


H0: Частота выбора у обоих баннеров одинаковая

H1: Один из баннеров выбирают чаще другого

Мы хотим проверить, насколько случайным является симпания тому или иному баннеру среди пользователей, то есть равна ли их вероятность 0.5
В таком случае применим критерий знаков, чтобы проверить распределение на биномиальное

источник : 7 семинар


In [25]:
a1 = df['banner1']
b2 = df['banner2']

from statsmodels.stats.descriptivestats import sign_test
print(f' results = {sign_test(a1-b2)}')

 results = (-8.0, 0.005222879350185395)


In [47]:
print(df['banner1'].value_counts(ascending=False))
print('\n')
print(df['banner2'].value_counts(ascending=False))


0    963
1     37
Name: banner1, dtype: int64


0    947
1     53
Name: banner2, dtype: int64


Итак, мы получили значение p value= 0.005, что свидетельствует о том, что экпериментальные данные противоречат H0 на любом уровне значимости и мы отвергаем ее в пользу H1

То есть, существует различие в восприятии баннеров пользователями и число их лайков в пользу второго баннера, который нравится пользователям в среднем больше. Однако стоит отметить, что разброс значений данного баннера также больше первого, вероятно он воспринимается более субъективно, возможно такой баннер содержит специфический юмор или же нестандартные для визуального восприятия цвета

## Задача 3. Немного АВ-тестов (3 балла)
## Игра Cookie Cat

В этом задании мы поработаем с данными Cookie Cats - популярной игры-головоломки, которую разработала компания Tactile Entertainment. В этой игре игрок должен соединять одинаковые элементы в ряд, чтобы очистить поле и выиграть уровень. 

<p style="align: center;"><img align=center src="https://i.ytimg.com/vi/iPxZIp0cbJE/maxresdefault.jpg"  width=500></p>
 
 
<p style="align: center;"><img align=center src="https://s3.amazonaws.com/assets.datacamp.com/production/project_184/img/cc_gates.png"  width=500></p>


По мере прохождения уровней перед игроками появляются ворота, открытия которых нужно ждать какое-то время или пропускать их, совершая покупки в приложении. С точки зрения создателей эти ворота повышают интерес к игре и стимулируют к вложению денег в игру.


Но у создателей появляется важный вопрос: **где лучше всего ставить ворота?**
Первоначально первые ворота были размещены на уровне 30. Однако есть гипотеза, что лучше переместить их на уровень 40. В этом задании вам предлагается проанализировать результаты AB-теста, в котором создатели переместили первые ворота в Cookie Cats с 30 уровня на уровень 40. 

**Данные получены от 90 189 игроков, которые установили игру во время проведения AB-теста. Переменные:**

* `userid` - уникальный номер, идентифицирующий каждого игрока.

* `version` - был ли игрок помещен в контрольную группу (gate_30 - ворота на уровне 30) или в тестовую группу (gate_40 - ворота на уровне 40).

* `sum_gamerounds` - количество игровых раундов, сыгранных игроком в течение первой недели после установки
* `retention_1` - вернулся ли игрок в игру через 1 день после установки
* `retention_7` - вернулся ли игрок в игру через 7 дней после установки

Когда игрок устанавливал игру, ему случайным образом назначали `gate_30` или `gate_40`.

In [18]:
import pandas as pd
df = pd.read_csv('cookie_cats.csv')
df.head()

,userid,version,sum_gamerounds,retention_1,retention_7
0,116,gate_30,3,False,False
1,337,gate_30,38,True,False
2,377,gate_40,165,True,False
3,483,gate_40,1,False,False
4,488,gate_40,179,True,True


In [19]:
df.describe()

,userid,sum_gamerounds
count,9.018900e+04,90189.000000
mean,4.998412e+06,51.872457
std,2.883286e+06,195.050858
min,1.160000e+02,0.000000
25%,2.512230e+06,5.000000
50%,4.995815e+06,16.000000
75%,7.496452e+06,51.000000
max,9.999861e+06,49854.000000


### Вопрос 1: есть ли зависимость между возвращаемостью игроков и расположением ворот?

#### 1.1 Проверьте гипотезу для возвращения через день

H0: Зависимости между возвращаемостью игроков через день и расположением ворот нет

H1: Зависимость между возвращаемостью игроков через день и расположением ворот есть

Так как обе наши переменные категориальные, используем критерий хи квадрат Пирсона на независимость двух признаков,  источник: семинар 7

In [20]:
### ╰( ͡° ͜ʖ ͡° )つ──☆ сотворите статистическую магию

t = df.groupby('version')['retention_1'].value_counts().unstack()

print(stats.chi2_contingency(t, correction=False))

(3.182963657512031, 0.07440965529692188, 1, array([[24799.13514952, 19900.86485048],
       [25236.86485048, 20252.13514952]]))


Так как получившийся p value =0.07, мы не отклоняем нулевую гипотезу на стандартном уровне значимости 5%(а также 1%), то есть на основе получившихся данных, можно заключить, что зависимости между возвращаемостью игроков через день и расположением ворот нет. То есть, вне заивисимости от расположеия ворот, средний игрок не возвращется через день чаще исходного

#### 2.2 Проверьте гипотезу для возвращения через семь дней

In [21]:
### ╰( ͡° ͜ʖ ͡° )つ──☆ сотворите статистическую магию

t = df.groupby('version')['retention_7'].value_counts().unstack()
print(stats.chi2_contingency(t, correction=False))

(10.013167328688969, 0.0015542499756142805, 1, array([[36382.90257127,  8317.09742873],
       [37025.09742873,  8463.90257127]]))


В данном же случае на основе получившегося p value=0.0016, мы отвергаем H0 на любом разумном уровне значимости и принимаем H1 о том, что зависимость между возвращаемостью игроков через семь дней и расположением ворот есть. То есть в случае перемещения ворот на уровень 40, пользователь возвращается в течение недели, значит такое размещение, в отличие от первого, способствует возвращению игроков, хотя и в течение недели

### Вопрос 2: Отличается ли среднее количество раундов в группах gate_40 и gate_30?

H0: Среднее количество раундов в двух группах одинаково

H1:Среднее количество раундов в двух группах отличается


Для работы с проверкой гипотезы о средних в числовых и категориальных данных, используем стандартный т критерий Стьюдента для параметрического критерия двух независимых выборок с двусторонней альтернативой(тк не сказано, в чью пользу отличается среднее)


In [22]:
### ╰( ͡° ͜ʖ ͡° )つ──☆ сотворите статистическую магию


gate_40_group = df[df.version == 'gate_40']['sum_gamerounds']
gate_30_group = df[df.version == 'gate_30']['sum_gamerounds']


from scipy import stats
ttest, pval = stats.ttest_ind(gate_40_group, gate_30_group ,alternative = 'two-sided')
print(f'stat={ttest}, p-value={pval}')

stat=-0.8910426211362967, p-value=0.37290868247405207


Итак, мы получили p value =0.37, следовательно мы не отклоняем нулевую гипотезу на любом разумном уровне значимости на основе имеющихся экспериментальных данных, то есть и тестовая и стандартная группа вне зависимости от расположения ворот играют в среднем одинаковое количество раундов

### Вопрос 3: Отличается ли медианное количество раундов в группах gate_40 и gate_30?

H0: Медианное количество раундов в двух группах одинаково

H1: Медианное количество раундов в двух группах отличается

Так как медиана относится к непараметрическим критериям, а наши выборки независимые, используем тест Манна-Уитн для проверки гипотезы о равенстве, выборки независимые с двусторонней альтернативой(тк не сказано, в чью пользу отличаются медианы), источник: 7 семинар 

In [30]:
### ╰( ͡° ͜ʖ ͡° )つ──☆ сотворите статистическую магию
x = df[df['version'] == 'gate_40']['sum_gamerounds']
y = df[df['version'] == 'gate_30']['sum_gamerounds']
x.name, y.name = 'gate_40', 'gate_30'
res = stats.mannwhitneyu(x, y)
res

MannwhitneyuResult(statistic=1009027049.5, pvalue=0.05020880772044255)

В данном случае значение p value=0.0502, что является несколько больше 5%, следовательно мы не отвергаем H0 на стандартном уровне значимости 5% и делаем вывод, что медианное количество раундов в двух группах равнозначное. 
Отличие между количеством сыгранных раундов у игроков не существенное, хотя стоит отметить, что при небольшом увеличении нашего порогового уровня значимости, медианные значения будут отличатся в пользу gate_30

В целом, можно сказать, что в краткосроной перспективе на пристрастие к игре участниками не влияет расположение ворот, однако в течение недели пользователи все таки продолжают игру, при этом в среднем с тем же количеством сыгранных раундов. 
Нам лучше разместить ворота на уровне 40, так как это оказывает положительную динамику во взаимодейсвтии с пользователем

## Задача 4: Время покемонов (3 балла)

В данной задаче вам будет необходимо проанализировать различные характеристики покемонов и сравнить их между собой

<p style="align: center;"><img align=center src="https://proprikol.ru/wp-content/uploads/2020/11/kartinki-pokemonov-2.jpg"  width=500></p>

Покемоны - это маленькие существа, которые сражаются друг с другом на соревнованиях. Все покемоны имеют разные характеристики (сила атаки, защиты и т. д.) и относятся к одному или двум так называемым классам (вода, огонь и т. д.). Профессор Оук является изобретателем Pokedex, портативного устройства, которое хранит информацию обо всех существующих покемонах. Как его ведущий специалист по данным, Вы только что получили от него запрос с просьбой осуществить аналитику данных на всех устройствах Pokedex.

### Описание данных

Профессор Оук скопировал все содержимое память одного устройства Pokedex, в результате чего получился набор данных, с которым Вы будете работать в этой задаче. В этом файле каждая строка представляет характеристики одного покемона:

* pid: Numeric - ID покемона
* HP: Numeric - Очки здоровья
* Attack: Numeric - Сила обычной атаки
* Defense: Numeric - Сила обычной защиты
* Sp. Atk: Numeric - Сила специальной атаки
* Sp. Def: Numeric - Сила специальной защиты
* Speed: Numeric - Скорость движений
* Legendary: Boolean - «True», если покемон редкий
* Class 1: Categorical - Класс покемона
* Class 2: Categorical - Класс покемона

In [12]:
import pandas as pd
df = pd.read_csv('pokemon.csv')
# для удобства работы в дальнейшем переименуем две колонки
df.rename(columns = {'Class 1':'xclass', 'Class 2':'yclass'}, inplace = True)
df.head()

,pid,Name,xclass,yclass,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Legendary
0,1,Bulbasaur,Grass,Poison,45,49,49,65,65,45,False
1,2,Ivysaur,Grass,Poison,60,62,63,80,80,60,False
2,3,Venusaur,Grass,Poison,80,82,83,100,100,80,False
3,4,Mega Venusaur,Grass,Poison,80,100,123,122,120,80,False
4,5,Charmander,Fire,NaN,39,52,43,60,50,65,False


### Вопрос 1

Профессор Оук подозревает, что покемоны в классе `grass` имеют более сильную обычную атаку, чем у покемонов в классе `rock`. Проверьте, прав ли он, и убедите его в своем выводе статистически. 

In [13]:
print(df['xclass'].unique())
print(df['yclass'].unique()) 

['Grass' 'Fire' 'Water' 'Bug' 'Normal' 'Poison' 'Electric' 'Ground'
 'Fairy' 'Fighting' 'Psychic' 'Rock' 'Ghost' 'Ice' 'Dragon' 'Dark' 'Steel'
 'Flying']
['Poison' nan 'Flying' 'Dragon' 'Ground' 'Fairy' 'Grass' 'Fighting'
 'Psychic' 'Steel' 'Ice' 'Rock' 'Dark' 'Water' 'Electric' 'Fire' 'Ghost'
 'Bug' 'Normal']


Видим, что нужные нам классы встречаются в обоих колонках, чтобы наличие у покемона нескольких классов не повлияло на наш результат, создадим новую колонку с покемонами, которые имеют только один класс(в нашем случае это только покемоны с 1 классом, так как эта ячейка заполнена в любом случае). Отметим также, что лучше не брать в учет наличие дублирующихся классов у одного покемона, так как в таком случае характеристики покемонов складываются, что может демонстрировать нерепрезентативные показатели и, следовательно, препятствовать сравнению силы атаки напрямую.

In [14]:
import numpy as np

df['oneclass'] = np.where(df['yclass']!= '', df['xclass'], False)
df.head()


,pid,Name,xclass,yclass,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Legendary,oneclass
0,1,Bulbasaur,Grass,Poison,45,49,49,65,65,45,False,Grass
1,2,Ivysaur,Grass,Poison,60,62,63,80,80,60,False,Grass
2,3,Venusaur,Grass,Poison,80,82,83,100,100,80,False,Grass
3,4,Mega Venusaur,Grass,Poison,80,100,123,122,120,80,False,Grass
4,5,Charmander,Fire,NaN,39,52,43,60,50,65,False,Fire



H0: Покемоны двух классов grass и rock имеют одинаковую силу обычной атаки

H1: Покемоны в классе grass имеют более сильную обычную атаку, чем у покемонов в классе rock
    
    
Так как мы снова имеем дело с данными типа numeric/ categorial, используем двухвыборочный тест ttest_ ind для независимых выборок
 с альтернативой greater(так как мы проверяем больше ли сила атаки у класса grass)


In [15]:
### ╰( ͡° ͜ʖ ͡° )つ──☆ сотворите статистическую магию

grass_group = df[df.oneclass == 'Grass']['Speed']
rock_group = df[df.oneclass == 'Rock']['Speed']


from scipy import stats
ttest, pval = stats.ttest_ind(grass_group, rock_group ,alternative = 'greater')
print(f'stat={ttest}, p-value={pval}')


if pval > 0.05:
    print('Покемоны двух классов grass и rock имеют одинаковую силу обычной атаки')
else:
    print('Покемоны в классе grass имеют более сильную обычную атаку, чем у покемоны в классе rock')

stat=1.077020239270595, p-value=0.14189280158888148
Покемоны двух классов grass и rock имеют одинаковую силу обычной атаки


Таким образом, получившийся p value= 0,14 свидетельсвует о том что экспрементальные данные не противоречат нулевой гипотезе на любом разумном уровне значимости, из-за чего мы не отвергаем H0: покемоны в классе grass имеют равносильную обычную атаку в сравнении с покемонами в классе rock при "чистом" сравнении классов друг с другом, то есть отсутствии у них других дополнительных классов либо при отсутствии дублирующихся классов у одного покемона.
То есть, класс покемонов grass не сильнее класса rock по отдельности

### Вопрос 2

Также профессор Оук очень хочет проверить, правда ли нет значимой разницы между силой обычной атаки и силой обычной защиты у покемонов. Помогите ему и ответьте на вопрос, подтвердив свои выводы с помощью проверки статистических гипотез

H0: Нет значимой разницы между силой обычной атаки и силой обычной защиты у покемонов

H1: Сила обычной атаки и сила обычной защиты у покемонов отличается

Так как мы работаем с зависимыми выборками числовой катерии даннных, воспользуемся парный т критерием стьюдента ttest_rel  с двусторонней альтернативой (upd: после двустороннего теста, мы хотим проверить, в чью пользу свидетельствуют экспериментальные данные, из-за чего в финальной версии присутствует альтернатива greater)    
    
источник: семинар 6

In [16]:
### ╰( ͡° ͜ʖ ͡° )つ──☆ сотворите статистическую магию
att=df['Attack']
de=df['Defense']
stats.ttest_rel(att,de, alternative ='greater')

Ttest_relResult(statistic=4.325566393330478, pvalue=8.570151739679279e-06)

Получившийся p value говорит нам о том, что на основе имеющихся данных на любом разумном уровне значимости мы отклоняем H0 и принимаем альтернативную гипотезу о том, что сила обычной атаки у покемонов отличается от силы обычной защиты. 
Применяя односторонний тест, можно убедиться, что сила атаки покемонов статистически больше силы защиты. 

Покемоны лучше показывают себя в нападении нежели в обороне

### Вопрос 3

И последний вопрос, который мучает профессора Оука: правда ли, что средняя скорость покемонов превышает 66.6 ? Сформулируйте подходящие гипотезы и помогите профессору Оуку, ответив на вопрос.

H0: Средняя скорость покемонов = 66.6

H1: Средняя скорость покемонов > 66.6

Так как мы сравниваем выборочное среднее с числом и при этом не знаем генеральное стандартное отклонение, используем т тест с альтернативой greater

семинар 5

In [87]:
### ╰( ͡° ͜ʖ ͡° )つ──☆ сотворите статистическую магию
from scipy.stats import ttest_1samp

ttest_1samp(df['Speed'], popmean = 66.6, alternative = 'greater')

Ttest_1sampResult(statistic=1.6326941356636653, pvalue=0.051463667549085566)

По итогу мы получили p value = 0.0515, из-за чего на стандартном уровне значимости 5% мы не отвергаем H0 и => наш вывод: В среднем скорость покемонов примерно составляет 66.6